In [111]:
import pandas as pd
import glob, os 
import pickle
import stdb
from obspy.geodetics.base import gps2dist_azimuth

In [61]:
path = '/Volumes/home/Research/STEP/02_Station_result_csv'
pklpath = '/Volumes/home/Research/STEP/01_Split'

previous = ['DGRL','CMCY','MAKU']

In [158]:
len(glob.glob(f'{path}/*SKS*classify/*result.csv'))

50

In [173]:
totaldf = pd.DataFrame()
for stapkl in sorted(glob.glob(f'{pklpath}/*pkl')):
    station = pickle.load(open(stapkl, "rb"))
    values = [value for keys, value in station.items() ][0]
    NET = values['network']; STA = values['station']
    stlat = values['latitude']; stlon=values['longitude']
    if STA not in previous:
        dff_ = pd.DataFrame()
        bazlist = []
        for csvpath in glob.glob(f'{path}/*classify/{NET}.{STA}*result.csv'):
            print(csvpath)
            df = pd.read_csv(csvpath)
            df_ = df[df['Pick']==True]
            dff_ = pd.concat([dff_,df_])
            
        for i in range(len(dff_)):
            evlon = dff_['Ev_lon'].values[i]
            evlat = dff_['Ev_lat'].values[i]
            dist, az, baz = gps2dist_azimuth(evlat, evlon, stlat,stlon)
            bazlist.append(round(baz,2))
        length=len(dff_)
        dff_ = dff_.drop(['SNRQ','SNRT','Pick'], axis=1)
        colname=dff_.columns.to_list()
        colname.insert(colname.index('Phase'),'Baz')
        colname.insert(colname.index('Event'),'Station')
        
        newdf = dff_.reindex(columns=colname)
        newdf['Station']=[f'{NET}.{STA}']*length
        newdf['Baz']=bazlist
        
        totaldf = pd.concat([totaldf,newdf],ignore_index=True)
        
totaldf.to_csv('catalog.csv')

/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify/AB.GANJ_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify/AB.GANJ_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify/AB.QZX_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify/AB.QZX_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify/AB.ZKT_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify/AB.ZKT_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify/AM.ALAV_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify/AM.ALAV_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify/AM.ARZA_split_result.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify/AM.ARZA_split_result.csv
/Volumes/home/Resea

In [174]:
!open .


In [175]:
totaldf

,Station,Event,Ev_lat,Ev_lon,Depth,Mag,Baz,Phase,SCPhi,SCPhi_std,SCdt,SCdt_std,RCPhi,RCPhi_std,RCdt,RCdt_std,Null,Quality,CpH,coeff
0,AB.GANJ,20110824_174611,-7.6203,-74.5380,149.3,7.0,285.36,SKS,-88.64,22.00,0.7,0.95,61.36,22.25,0.3,0.98,True,Fair,0.87,-0.83
1,AB.GANJ,20110830_065741,-6.3592,126.7502,467.2,6.9,100.91,SKS,-24.09,21.50,0.8,0.95,-35.09,44.75,0.7,0.98,False,Fair,0.73,1.00
2,AB.GANJ,20110903_044858,-56.5611,-26.9231,96.3,6.4,215.73,SKS,-59.27,44.75,2.9,0.98,71.73,44.75,0.3,0.98,True,Fair,0.85,1.00
3,AB.GANJ,20110909_194133,49.3886,-127.0648,15.4,6.5,355.71,SKS,-64.29,44.75,0.6,0.98,-44.29,44.75,0.5,0.98,False,Fair,0.59,1.00
4,AB.GANJ,20111014_033515,-6.6259,147.9311,43.2,6.5,87.33,SKS,-44.67,13.25,0.8,0.18,-47.67,18.00,0.8,0.35,False,Good,0.70,-0.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544,TG.URAV,20110620_163600,-21.7710,-68.3834,126.2,6.5,267.51,SKKS,1.51,1.25,3.9,0.48,48.51,44.75,0.1,0.98,True,Fair,0.82,1.00
1545,TG.URAV,20110729_074222,-23.7284,179.8252,522.8,6.7,75.98,SKKS,78.98,44.75,3.9,0.98,33.98,44.75,0.0,0.98,True,Good,0.95,1.00
1546,TG.URAV,20120109_040715,-10.5892,165.3084,33.3,6.4,75.15,SKKS,-20.85,23.25,3.9,0.38,-68.85,44.75,0.1,0.98,True,Fair,0.89,1.00
1547,TG.URAV,20120124_005204,-24.8986,178.5999,579.0,6.3,78.46,SKKS,87.46,44.75,3.7,0.98,-42.54,44.75,0.1,0.98,True,Fair,0.80,1.00
